In [119]:
import requests

In [122]:
sys = """Objective: Create a detailed list of skills from the provided NOS and associate each skill with assessment criteria aligned to Bloom's Taxonomy (6 levels: Remember, Understand, Apply, Analyze, Evaluate, Create). This will ensure meaningful skill assessment for both technical and soft skills.

Instructions for the System:

Extract Skills:

Analyze the provided NOS document and extract all the skills mentioned. For each skill, ensure both technical (e.g., product knowledge, technical tools) and soft skills (e.g., communication, customer engagement) are included.
Each skill should be contextualized based on the job description (JD) and performance standards outlined in the NOS.
Define Assessment Criteria:

For each skill, generate assessment criteria based on the six levels of Bloom’s Taxonomy:
Remember: Does the user recall relevant facts, definitions, or procedures for this skill?
Understand: Can the user explain or interpret the concepts related to this skill?
Apply: Can the user use the skill in real-world scenarios or simulated tasks?
Analyze: Can the user break down complex situations to identify components related to this skill?
Evaluate: Can the user assess situations and justify decisions involving this skill?
Create: Can the user design or innovate new approaches, presentations, or solutions based on this skill?
Associate Performance and Knowledge Criteria:

Align each skill with performance standards and knowledge requirements from the NOS. This ensures the assessment is comprehensive and role-specific.
Include prompts for assessing both practical demonstration and theoretical understanding.
Output Format:

For each skill, provide:
Skill Name
Skill Category (Technical/Soft)
Bloom's Taxonomy Level Criteria:
Remember: Specific questions or tasks to test recall.
Understand: Scenarios to explain the skill.
Apply: Practical tasks or case studies.
Analyze: Problem-solving scenarios.
Evaluate: Judgment or decision-making tasks.
Create: Designing new solutions or improvements.

"""

In [123]:
system_message = {'role':'system','content':sys}
nos = """Performance criteria
You must be able to:
1. use sales-related information for planning and implementing sales activities
2. store information on the sales technology systems, checking its quality, validity
and reliability
3. maintain the sales-related information stored in the technology systems, ensuring
it is accurate and up to date
4. use tools and technology systems for automation of sales-related tasks and
activities
5. share sales-related information for collaboration with members of sales team
6. use lead generation automation technology to qualify the leads
7. collate potential leads and prioritise these in accordance with sales targets
8. summarise information appropriately from sales systems for reporting to
management
9. follow up incoming leads and assign the ownership of quality leads to the most
relevant members of sales team
10. monitor, process or delegate daily tasks within assigned ownership and visibility on
technology systems
11. monitor own use of the sales technology system, seeking guidance, support and
skills development, where necessary
12. provide feedback on the sales technology systems, your pipeline management and
other tasks and activities
13. seek evidence-based recommendations for improvement of sales activities and
personal productivity using the technology systems
INSSAL014
Use sales tools and technology systems
INSSAL014 Use sales tools and technology systems 2
Knowledge and understanding
You need to know and understand:
1. the current legal and ethical issues in relation to the use of sales tools and
technology systems
2. the benefits of tools and technology systems for sales-related tasks and activities
3. the range of tasks and activities on sales tools and technology systems
4. the visibility options and ownership of sales team members on technology systems
5. the information requirements for the sales function, lead generation and
qualification of these
6. your organisation’s sales targets
7. the importance of automation of sales-related tasks and activities
8. how to collate, qualify and prioritise the leads on technology systems
9. the sales pipeline and how to feed the leads into it using technology systems
10. how to maintain and monitor your sales pipeline
11. when to assign the leads to the relevant members of sales team or to yourself
12. your organisation’s processes and procedures for using, storing and maintaining 
information
13. how to use the sales technology system as part of planning and implementation of
sales activities
14. the support, guidance and skills development available to the users of sales tools
and technology system
15. how to seek recommendations for improvements of sales activities using the tools
and technology systems"""

In [124]:
from pydantic import Field
from typing import Dict, List
from helpers._types import (
    Agent,
    StrictTool,
    Result,
)

In [125]:
import os
os.chdir(f"{os.path.dirname(os.path.abspath(os.getcwd()))}/zavmo")

In [126]:
import openai
import json
from openai import OpenAI
from pydantic import BaseModel, Field
from typing import List, Optional, Literal
# from IPython.display import Markdown

In [143]:
class BloomTaxonomyLevels(StrictTool):
    level: Literal["Remember", "Understand", "Apply", "Analyze", "Evaluate", "Create"] = Field(description="The level of Bloom's Taxonomy")
    criteria: str = Field(description="Generate very challenging criteria for assessing the skill based on shared NOS for the level of Bloom's Taxonomy")

    def execute(self, context: Dict):
        return self.model_dump_json()
    
class GetSkillFromNOS(StrictTool):
    competency:str = Field(description="Name of the skill area")
    blooms_taxonomy_criteria: List[BloomTaxonomyLevels] = Field(description="Criteria for Bloom's Taxonomy levels")
    
    def execute(self, context: Dict):
        return self.model_dump_json()

class GetSkillsFromNOS(StrictTool):
    """A tool to extract multiple skills names, categories and assessment criteria from the NOS document."""
    
    nos: List[GetSkillFromNOS] = Field(description="List of skills with categories and criteria on Bloom's Taxonomy levels")
    
    def execute(self, context: Dict):
        return self.model_dump_json()


In [137]:
from helpers.swarm import fetch_agent_response


In [144]:
messages = [{"role": "user", "content": f"Here is the NOS(National Occupational Standards) document: {nos}"}]
agent = Agent(
    name="GetSkillsFromNOS",
    functions=[GetSkillsFromNOS],
    model="gpt-4o-mini",
    system_message=system_message,
)

agent.instructions = sys

response = fetch_agent_response(agent, messages, {})  

2024-12-06 15:47:42,030 - INFO - Logging from fetch_agent response


In [146]:
response_content = json.loads(response.choices[0].message.tool_calls[0].function.arguments)
response_content

{'nos': [{'competency': 'Use sales-related information for planning and implementing sales activities',
   'blooms_taxonomy_criteria': [{'level': 'Remember',
     'criteria': 'List the steps involved in using sales-related information for planning.'},
    {'level': 'Understand',
     'criteria': 'Explain how sales information impacts decision-making in planning sales activities.'},
    {'level': 'Apply',
     'criteria': 'Create a sales plan using sales-related information for a hypothetical product.'},
    {'level': 'Analyze',
     'criteria': 'Evaluate the effectiveness of a sales plan based on provided sales data.'},
    {'level': 'Evaluate',
     'criteria': 'Assess the outcomes of a sales activity against planning objectives.'},
    {'level': 'Create',
     'criteria': 'Design a comprehensive sales planning process based on current sales data.'}]},
  {'competency': 'Store information on the sales technology systems, checking its quality, validity and reliability',
   'blooms_taxon

In [147]:
with open(os.path.join(os.getcwd(), "assets/nos/sales/INSSAL014.json"), 'w') as file:
    json.dump(response_content['nos'], file, indent=2)

In [150]:
def get_nos_skills_with_criteria():
    nos_skills = response_content['nos']
    skill_criteria_str = ""
    for skill in nos_skills:
        skill_name = skill.get("competency")
        criteria = skill.get("blooms_taxonomy_criteria")
        criteria_str = "\n"+"\n".join([f"{item['level']}: {item['criteria']}" for item in criteria])
        skill_criteria_str += f"Competency: {skill_name}\nCriteria: {criteria_str}\n\n"
    return skill_criteria_str

In [151]:
print(get_nos_skills_with_criteria())

Competency: Use sales-related information for planning and implementing sales activities
Criteria: 
Remember: List the steps involved in using sales-related information for planning.
Understand: Explain how sales information impacts decision-making in planning sales activities.
Apply: Create a sales plan using sales-related information for a hypothetical product.
Analyze: Evaluate the effectiveness of a sales plan based on provided sales data.
Evaluate: Assess the outcomes of a sales activity against planning objectives.
Create: Design a comprehensive sales planning process based on current sales data.

Competency: Store information on the sales technology systems, checking its quality, validity and reliability
Criteria: 
Remember: Identify the key elements necessary to ensure data quality in sales systems.
Understand: Describe the implications of storing unreliable data in sales technology systems.
Apply: Implement a data validation process in a sales technology system.
Analyze: Break

In [ ]:
competencies = [
  {
    "competency": "Use sales-related information for planning and implementing sales activities",
    "blooms_taxonomy_criteria": [
      {
        "level": "Remember",
        "criteria": "List the steps involved in using sales-related information for planning."
      },
      {
        "level": "Understand",
        "criteria": "Explain how sales information impacts decision-making in planning sales activities."
      },
      {
        "level": "Apply",
        "criteria": "Create a sales plan using sales-related information for a hypothetical product."
      },
      {
        "level": "Analyze",
        "criteria": "Evaluate the effectiveness of a sales plan based on provided sales data."
      },
      {
        "level": "Evaluate",
        "criteria": "Assess the outcomes of a sales activity against planning objectives."
      },
      {
        "level": "Create",
        "criteria": "Design a comprehensive sales planning process based on current sales data."
      }
    ]
  },
  {
    "competency": "Store information on the sales technology systems, checking its quality, validity and reliability",
    "blooms_taxonomy_criteria": [
      {
        "level": "Remember",
        "criteria": "Identify the key elements necessary to ensure data quality in sales systems."
      },
      {
        "level": "Understand",
        "criteria": "Describe the implications of storing unreliable data in sales technology systems."
      },
      {
        "level": "Apply",
        "criteria": "Implement a data validation process in a sales technology system."
      },
      {
        "level": "Analyze",
        "criteria": "Break down the factors that contribute to data reliability in sales systems."
      },
      {
        "level": "Evaluate",
        "criteria": "Appraise the quality of information stored in a given sales technology case."
      },
      {
        "level": "Create",
        "criteria": "Develop a framework for assessing the quality and validity of data in sales systems."
      }
    ]
  },
  {
    "competency": "Maintain the sales-related information stored in the technology systems, ensuring it is accurate and up to date",
    "blooms_taxonomy_criteria": [
      {
        "level": "Remember",
        "criteria": "Recall the standards for maintaining accurate sales information."
      },
      {
        "level": "Understand",
        "criteria": "Clarify the process for updating information in sales technology systems."
      },
      {
        "level": "Apply",
        "criteria": "Demonstrate updating and maintaining sales information in a provided technology system scenario."
      },
      {
        "level": "Analyze",
        "criteria": "Examine discrepancies in sales records and identify maintenance needs."
      },
      {
        "level": "Evaluate",
        "criteria": "Judge the effectiveness of the current procedures for maintaining sales-related information."
      },
      {
        "level": "Create",
        "criteria": "Formulate a guideline document for maintaining accurate sales information."
      }
    ]
  },
  {
    "competency": "Use tools and technology systems for automation of sales-related tasks and activities",
    "blooms_taxonomy_criteria": [
      {
        "level": "Remember",
        "criteria": "List different tools available for automating sales tasks."
      },
      {
        "level": "Understand",
        "criteria": "Explain the advantages of automation in sales-related activities."
      },
      {
        "level": "Apply",
        "criteria": "Utilize an automation tool to perform a sales-related task."
      },
      {
        "level": "Analyze",
        "criteria": "Compare automated processes versus manual processes in sales activities."
      },
      {
        "level": "Evaluate",
        "criteria": "Assess the impact of automation on sales team productivity."
      },
      {
        "level": "Create",
        "criteria": "Design an automated workflow for a specific sales task."
      }
    ]
  },
  {
    "competency": "Share sales-related information for collaboration with members of the sales team",
    "blooms_taxonomy_criteria": [
      {
        "level": "Remember",
        "criteria": "Enumerate methods for sharing sales information with team members."
      },
      {
        "level": "Understand",
        "criteria": "Discuss the importance of collaboration in sales activities."
      },
      {
        "level": "Apply",
        "criteria": "Collaborate with a team by sharing feedback on sales prospects using a shared platform."
      },
      {
        "level": "Analyze",
        "criteria": "Identify barriers to effective information sharing in a sales team."
      },
      {
        "level": "Evaluate",
        "criteria": "Critique a case study on collaboration within a sales team."
      },
      {
        "level": "Create",
        "criteria": "Craft a collaborative communication strategy for a sales team."
      }
    ]
  },
  {
    "competency": "Use lead generation automation technology to qualify the leads",
    "blooms_taxonomy_criteria": [
      {
        "level": "Remember",
        "criteria": "Identify the key features of lead generation automation technology."
      },
      {
        "level": "Understand",
        "criteria": "Explain how automation improves the qualification process of leads."
      },
      {
        "level": "Apply",
        "criteria": "Use a lead generation tool to qualify a list of potential leads."
      },
      {
        "level": "Analyze",
        "criteria": "Assess the quality of qualified leads generated through automation."
      },
      {
        "level": "Evaluate",
        "criteria": "Evaluate the effectiveness of different lead generation methods."
      },
      {
        "level": "Create",
        "criteria": "Develop a new strategy for lead qualification using automation tools."
      }
    ]
  },
  {
    "competency": "Collate potential leads and prioritise these in accordance with sales targets",
    "blooms_taxonomy_criteria": [
      {
        "level": "Remember",
        "criteria": "State the criteria for prioritizing leads according to sales targets."
      },
      {
        "level": "Understand",
        "criteria": "Interpret sales targets and their implications for lead prioritization."
      },
      {
        "level": "Apply",
        "criteria": "Prioritize a given list of leads based on specified sales targets."
      },
      {
        "level": "Analyze",
        "criteria": "Analyze the given sales data to identify high-priority leads."
      },
      {
        "level": "Evaluate",
        "criteria": "Review the effectiveness of a lead prioritization strategy."
      },
      {
        "level": "Create",
        "criteria": "Design a lead prioritization framework based on sales objectives."
      }
    ]
  },
  {
    "competency": "Summarise information appropriately from sales systems for reporting to management",
    "blooms_taxonomy_criteria": [
      {
        "level": "Remember",
        "criteria": "Recall the main components required in a sales report."
      },
      {
        "level": "Understand",
        "criteria": "Discuss the importance of clear reporting to management."
      },
      {
        "level": "Apply",
        "criteria": "Generate a report from sales data using a provided template."
      },
      {
        "level": "Analyze",
        "criteria": "Break down sales data to identify key insights for reports."
      },
      {
        "level": "Evaluate",
        "criteria": "Assess the clarity and effectiveness of a sales report."
      },
      {
        "level": "Create",
        "criteria": "Develop a comprehensive reporting template for sales data."
      }
    ]
  },
  {
    "competency": "Follow up incoming leads and assign the ownership of quality leads to the most relevant members of the sales team",
    "blooms_taxonomy_criteria": [
      {
        "level": "Remember",
        "criteria": "List the steps involved in lead follow-up processes."
      },
      {
        "level": "Understand",
        "criteria": "Explain the rationale behind assigning leads to specific team members."
      },
      {
        "level": "Apply",
        "criteria": "Execute a follow-up process for incoming leads using a CRM system."
      },
      {
        "level": "Analyze",
        "criteria": "Differentiate between high-quality and low-quality leads and their implications for ownership assignment."
      },
      {
        "level": "Evaluate",
        "criteria": "Examine a case where lead follow-up and assignment affected sales outcomes."
      },
      {
        "level": "Create",
        "criteria": "Formulate a policy for lead assignment and follow-up in a sales team."
      }
    ]
  },
  {
    "competency": "Monitor, process or delegate daily tasks within assigned ownership and visibility on technology systems",
    "blooms_taxonomy_criteria": [
      {
        "level": "Remember",
        "criteria": "Identify daily tasks typically monitored within a sales team."
      },
      {
        "level": "Understand",
        "criteria": "Explain the role of visibility in task delegation."
      },
      {
        "level": "Apply",
        "criteria": "Demonstrate monitoring daily tasks in a given sales technology system."
      },
      {
        "level": "Analyze",
        "criteria": "Evaluate the effectiveness of current monitoring practices for task management."
      },
      {
        "level": "Evaluate",
        "criteria": "Assess how effective delegation impacts team productivity."
      },
      {
        "level": "Create",
        "criteria": "Design a task monitoring and delegation strategy for sales activities."
      }
    ]
  },
  {
    "competency": "Monitor own use of the sales technology system, seeking guidance, support and skills development where necessary",
    "blooms_taxonomy_criteria": [
      {
        "level": "Remember",
        "criteria": "Recall the importance of self-monitoring in technology system usage."
      },
      {
        "level": "Understand",
        "criteria": "Identify how seeking guidance contributes to skill development."
      },
      {
        "level": "Apply",
        "criteria": "Reflect on personal experiences to identify areas for improvement in using sales technology."
      },
      {
        "level": "Analyze",
        "criteria": "Assess the effectiveness of personal monitoring strategies in using the sales technology system."
      },
      {
        "level": "Evaluate",
        "criteria": "Critique personal usage of sales technology systems based on performance outcomes."
      },
      {
        "level": "Create",
        "criteria": "Develop a personal development plan for improving skills in using sales technology."
      }
    ]
  },
  {
    "competency": "Provide feedback on the sales technology systems, your pipeline management and other tasks and activities",
    "blooms_taxonomy_criteria": [
      {
        "level": "Remember",
        "criteria": "List the types of feedback typically provided on sales systems."
      },
      {
        "level": "Understand",
        "criteria": "Explain the role of feedback in improving sales technology systems."
      },
      {
        "level": "Apply",
        "criteria": "Provide constructive feedback based on a scenario involving the sales system."
      },
      {
        "level": "Analyze",
        "criteria": "Examine the feedback mechanisms in place for sales technology systems."
      },
      {
        "level": "Evaluate",
        "criteria": "Assess the impact of feedback on the improvement of sales processes."
      },
      {
        "level": "Create",
        "criteria": "Propose a feedback system for enhancing sales technology effectiveness."
      }
    ]
  },
  {
    "competency": "Seek evidence-based recommendations for improvement of sales activities and personal productivity using the technology systems",
    "blooms_taxonomy_criteria": [
      {
        "level": "Remember",
        "criteria": "Recall a method for gathering evidence-based data."
      },
      {
        "level": "Understand",
        "criteria": "Describe how recommendations can enhance sales processes."
      },
      {
        "level": "Apply",
        "criteria": "Use data to formulate recommendations for improving sales activities."
      },
      {
        "level": "Analyze",
        "criteria": "Identify trends in sales data that can suggest areas for improvement."
      },
      {
        "level": "Evaluate",
        "criteria": "Assess the feasibility of implementing a given recommendation for improvement."
      },
      {
        "level": "Create",
        "criteria": "Develop an innovative approach for enhancing personal productivity using technology."
      }
    ]
  }
]

def parse_competencies(competencies):
    parsed_data = {}
    for competency in competencies:
        competency_name = competency["competency"]
        criteria = [item["criteria"] for item in competency["blooms_taxonomy_criteria"]]
        parsed_data[competency_name] = criteria
    return parsed_data

parsed_competencies = parse_competencies(competencies)
print(parsed_competencies)

In [24]:
url = 'http://127.0.0.1:8000/api/create_org/'


In [25]:
payload = {
    "org_name": "Kenpath"
}

In [26]:
response = requests.post(url, json=payload)

In [27]:
response

<Response [201]>

In [34]:
url = 'http://127.0.0.1:8000/api/create_user/'  
data = {
    'email': 'smrithi.shaji@kenpath.io',
    'org_id': '2'  
}

response = requests.post(url, json=data)
print(response.status_code)
print(response.json())


400
{'org_id': ['This field is required.']}
